# ROI: Madhya Pradesh
##  Export LST

In [ ]:
import ee
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import numpy as np
import calendar

# Initialize Earth Engine
ee.Authenticate()
ee.Initialize(project='ee-geosynta')

def get_mp_boundary():
    """Get Madhya Pradesh's administrative boundary"""
    states = ee.FeatureCollection('FAO/GAUL/2015/level1')
    mp = states.filter(ee.Filter.eq('ADM1_NAME', 'Madhya Pradesh')).geometry()
    return mp

def calculate_monthly_lst():
    """Calculate monthly LST averages for Madhya Pradesh"""
    modis_lst = ee.ImageCollection("MODIS/061/MOD11A2")
    roi = get_mp_boundary()

    current_year = datetime.now().year
    current_month = datetime.now().month

    monthly_data = []

    for year in range(2015, current_year + 1):
        for month in range(1, 13):
            if year == current_year and month > current_month:
                continue

            try:
                print(f"Processing monthly data for {calendar.month_name[month]} {year}...")

                monthly_collection = modis_lst.filter(ee.Filter.calendarRange(year, year, 'year')) \
                                            .filter(ee.Filter.calendarRange(month, month, 'month')) \
                                            .select('LST_Day_1km') \
                                            .map(lambda img: img.updateMask(img.gt(7500)))

                num_images = monthly_collection.size().getInfo()
                print(f"Number of images: {num_images}")

                if num_images == 0:
                    print(f"No data available")
                    continue

                monthly_mean = monthly_collection.mean()
                monthly_mean = monthly_mean.multiply(0.02).subtract(273.15)

                # Calculate additional statistics
                stats = monthly_mean.reduceRegion(
                    reducer=ee.Reducer.mean()
                            .combine(ee.Reducer.stdDev(), '', True)
                            .combine(ee.Reducer.minMax(), '', True),
                    geometry=roi,
                    scale=1000,
                    maxPixels=1e9
                ).getInfo()

                if 'LST_Day_1km_mean' in stats and stats['LST_Day_1km_mean'] is not None:
                    monthly_data.append({
                        'Year': year,
                        'Month': month,
                        'Month_Name': calendar.month_name[month],
                        'Mean_Temperature_Celsius': round(stats['LST_Day_1km_mean'], 2),
                        'Min_Temperature_Celsius': round(stats['LST_Day_1km_min'], 2),
                        'Max_Temperature_Celsius': round(stats['LST_Day_1km_max'], 2),
                        'StdDev_Temperature_Celsius': round(stats['LST_Day_1km_stdDev'], 2)
                    })
                    print(f"Processed: Mean={monthly_data[-1]['Mean_Temperature_Celsius']}°C, "
                          f"Min={monthly_data[-1]['Min_Temperature_Celsius']}°C, "
                          f"Max={monthly_data[-1]['Max_Temperature_Celsius']}°C")

            except Exception as e:
                print(f"Error processing {calendar.month_name[month]} {year}: {str(e)}")

    return pd.DataFrame(monthly_data)

def calculate_annual_lst():
    """Calculate annual LST averages for Madhya Pradesh"""
    modis_lst = ee.ImageCollection("MODIS/061/MOD11A2")
    roi = get_mp_boundary()

    current_year = datetime.now().year
    annual_data = []

    for year in range(2015, current_year + 1):
        try:
            print(f"Processing annual data for {year}...")

            yearly_collection = modis_lst.filter(ee.Filter.calendarRange(year, year, 'year')) \
                                       .select('LST_Day_1km') \
                                       .map(lambda img: img.updateMask(img.gt(7500)))

            if yearly_collection.size().getInfo() == 0:
                print(f"No data available")
                continue

            yearly_mean = yearly_collection.mean()
            yearly_mean = yearly_mean.multiply(0.02).subtract(273.15)

            # Calculate annual statistics
            stats = yearly_mean.reduceRegion(
                reducer=ee.Reducer.mean()
                        .combine(ee.Reducer.stdDev(), '', True)
                        .combine(ee.Reducer.minMax(), '', True),
                geometry=roi,
                scale=1000,
                maxPixels=1e9
            ).getInfo()

            if 'LST_Day_1km_mean' in stats and stats['LST_Day_1km_mean'] is not None:
                annual_data.append({
                    'Year': year,
                    'Mean_Temperature_Celsius': round(stats['LST_Day_1km_mean'], 2),
                    'Min_Temperature_Celsius': round(stats['LST_Day_1km_min'], 2),
                    'Max_Temperature_Celsius': round(stats['LST_Day_1km_max'], 2),
                    'StdDev_Temperature_Celsius': round(stats['LST_Day_1km_stdDev'], 2)
                })
                print(f"Processed: Mean={annual_data[-1]['Mean_Temperature_Celsius']}°C")

        except Exception as e:
            print(f"Error processing {year}: {str(e)}")

    return pd.DataFrame(annual_data)

def export_data():
    """Export both monthly and annual LST data for Madhya Pradesh"""
    try:
        # Calculate and export monthly data
        print("\nProcessing monthly averages...")
        monthly_df = calculate_monthly_lst()
        monthly_output = 'mp_monthly_lst_data.csv'
        monthly_df.to_csv(monthly_output, index=False)
        print(f"Monthly data exported to {monthly_output}")
        print("\nMonthly data sample:")
        print(monthly_df.head())

        # Calculate and export annual data
        print("\nProcessing annual averages...")
        annual_df = calculate_annual_lst()
        annual_output = 'mp_annual_lst_data.csv'
        annual_df.to_csv(annual_output, index=False)
        print(f"Annual data exported to {annual_output}")
        print("\nAnnual data:")
        print(annual_df)

        # Generate summary statistics
        print("\nMonthly Summary Statistics:")
        monthly_stats = monthly_df.groupby('Month_Name')['Mean_Temperature_Celsius'].agg(['mean', 'min', 'max', 'std']).round(2)
        print(monthly_stats)

        print("\nAnnual Summary Statistics:")
        annual_stats = annual_df['Mean_Temperature_Celsius'].agg(['mean', 'min', 'max', 'std']).round(2)
        print(annual_stats)

    except Exception as e:
        print(f"An error occurred: {str(e)}")
        print("Please check:")
        print("1. Earth Engine authentication is valid")
        print("2. Internet connection is stable")
        print("3. The specified region has available data")

if __name__ == "__main__":
    export_data()


Processing monthly averages...
Processing monthly data for January 2015...
Number of images: 4
Processed: Mean=23.08°C, Min=14.61°C, Max=34.28°C
Processing monthly data for February 2015...
Number of images: 4
Processed: Mean=29.23°C, Min=19.51°C, Max=41.5°C
Processing monthly data for March 2015...
Number of images: 4
Processed: Mean=36.27°C, Min=22.84°C, Max=47.17°C
Processing monthly data for April 2015...
Number of images: 3
Processed: Mean=41.25°C, Min=25.08°C, Max=50.12°C
Processing monthly data for May 2015...
Number of images: 4
Processed: Mean=46.0°C, Min=27.49°C, Max=53.25°C
Processing monthly data for June 2015...
Number of images: 4
Processed: Mean=40.56°C, Min=23.91°C, Max=54.03°C
Processing monthly data for July 2015...
Number of images: 4
Processed: Mean=33.56°C, Min=10.69°C, Max=47.19°C
Processing monthly data for August 2015...
Number of images: 4
Processed: Mean=30.23°C, Min=16.93°C, Max=41.57°C
Processing monthly data for September 2015...
Number of images: 4
Proces

## Export Humidity

In [ ]:
import ee
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import numpy as np
import calendar

# Initialize Earth Engine
ee.Authenticate()
ee.Initialize(project='ee-geosynta')

def get_mp_boundary():
    """Get Madhya Pradesh's administrative boundary"""
    states = ee.FeatureCollection('FAO/GAUL/2015/level1')
    mp = states.filter(ee.Filter.eq('ADM1_NAME', 'Madhya Pradesh')).geometry()
    return mp

def calculate_relative_humidity(T, p, q):
    """
    Calculate relative humidity using specific humidity, pressure, and temperature.
    Based on the August-Roche-Magnus approximation.

    Args:
        T: Temperature in Kelvin
        p: Surface pressure in Pa
        q: Specific humidity in kg/kg
    Returns:
        Relative humidity (0-1)
    """
    # Convert temperature to Celsius
    T_C = T.subtract(273.15)

    # Calculate saturation vapor pressure (hPa)
    es = ee.Image(6.112).multiply(ee.Image.exp(ee.Image(17.67).multiply(T_C).divide(T_C.add(243.5))))

    # Convert pressure from Pa to hPa
    p_hPa = p.divide(100)

    # Calculate vapor pressure from specific humidity
    e = q.multiply(p_hPa).divide(ee.Image(0.622).add(q.multiply(ee.Image(0.378))))

    # Calculate relative humidity (0-1)
    rh = e.divide(es)

    # Ensure values are between 0 and 1
    rh = rh.max(ee.Image(0)).min(ee.Image(1))

    return rh

def calculate_monthly_humidity():
    """Calculate monthly humidity averages for Madhya Pradesh"""
    gldas = ee.ImageCollection("NASA/GLDAS/V021/NOAH/G025/T3H")
    roi = get_mp_boundary()

    current_year = datetime.now().year
    current_month = datetime.now().month

    monthly_data = []

    for year in range(2015, current_year + 1):
        for month in range(1, 13):
            if year == current_year and month > current_month:
                continue

            try:
                print(f"Processing monthly data for {calendar.month_name[month]} {year}...")

                monthly_collection = gldas.filter(ee.Filter.calendarRange(year, year, 'year')) \
                                       .filter(ee.Filter.calendarRange(month, month, 'month')) \
                                       .select(['Tair_f_inst', 'Psurf_f_inst', 'Qair_f_inst'])

                num_images = monthly_collection.size().getInfo()
                print(f"Number of images: {num_images}")

                if num_images == 0:
                    print(f"No data available")
                    continue

                # Calculate means for each parameter
                T = monthly_collection.select('Tair_f_inst').mean()
                p = monthly_collection.select('Psurf_f_inst').mean()
                q = monthly_collection.select('Qair_f_inst').mean()

                # Calculate relative humidity
                rh = calculate_relative_humidity(T, p, q)

                # Combine all bands for statistics
                combined = T.addBands(p).addBands(q).addBands(rh.rename('RH'))

                stats = combined.reduceRegion(
                    reducer=ee.Reducer.mean()
                            .combine(ee.Reducer.stdDev(), '', True)
                            .combine(ee.Reducer.minMax(), '', True),
                    geometry=roi,
                    scale=25000,
                    maxPixels=1e9
                ).getInfo()

                if stats['Tair_f_inst_mean'] is not None:
                    monthly_data.append({
                        'Year': year,
                        'Month': month,
                        'Month_Name': calendar.month_name[month],
                        'Temperature_K': round(stats['Tair_f_inst_mean'], 2),
                        'Temperature_C': round(stats['Tair_f_inst_mean'] - 273.15, 2),
                        'Pressure_Pa': round(stats['Psurf_f_inst_mean'], 2),
                        'Specific_Humidity': round(stats['Qair_f_inst_mean'], 4),
                        'Relative_Humidity': round(stats['RH_mean'] * 100, 2),  # Convert to percentage
                        'Temperature_StdDev': round(stats['Tair_f_inst_stdDev'], 2),
                        'Pressure_StdDev': round(stats['Psurf_f_inst_stdDev'], 2),
                        'Specific_Humidity_StdDev': round(stats['Qair_f_inst_stdDev'], 4)
                    })
                    print(f"Processed: Temp={monthly_data[-1]['Temperature_C']}°C, RH={monthly_data[-1]['Relative_Humidity']}%")

            except Exception as e:
                print(f"Error processing {calendar.month_name[month]} {year}: {str(e)}")

    return pd.DataFrame(monthly_data)

def calculate_annual_humidity():
    """Calculate annual humidity averages for Madhya Pradesh"""
    gldas = ee.ImageCollection("NASA/GLDAS/V021/NOAH/G025/T3H")
    roi = get_mp_boundary()

    current_year = datetime.now().year
    annual_data = []

    for year in range(2015, current_year + 1):
        try:
            print(f"Processing annual data for {year}...")

            yearly_collection = gldas.filter(ee.Filter.calendarRange(year, year, 'year')) \
                                   .select(['Tair_f_inst', 'Psurf_f_inst', 'Qair_f_inst'])

            if yearly_collection.size().getInfo() == 0:
                print(f"No data available")
                continue

            # Calculate means for each parameter
            T = yearly_collection.select('Tair_f_inst').mean()
            p = yearly_collection.select('Psurf_f_inst').mean()
            q = yearly_collection.select('Qair_f_inst').mean()

            # Calculate relative humidity
            rh = calculate_relative_humidity(T, p, q)

            # Combine all bands for statistics
            combined = T.addBands(p).addBands(q).addBands(rh.rename('RH'))

            stats = combined.reduceRegion(
                reducer=ee.Reducer.mean()
                        .combine(ee.Reducer.stdDev(), '', True)
                        .combine(ee.Reducer.minMax(), '', True),
                geometry=roi,
                scale=25000,
                maxPixels=1e9
            ).getInfo()

            if stats['Tair_f_inst_mean'] is not None:
                annual_data.append({
                    'Year': year,
                    'Temperature_K': round(stats['Tair_f_inst_mean'], 2),
                    'Temperature_C': round(stats['Tair_f_inst_mean'] - 273.15, 2),
                    'Pressure_Pa': round(stats['Psurf_f_inst_mean'], 2),
                    'Specific_Humidity': round(stats['Qair_f_inst_mean'], 4),
                    'Relative_Humidity': round(stats['RH_mean'] * 100, 2),  # Convert to percentage
                    'Temperature_StdDev': round(stats['Tair_f_inst_stdDev'], 2),
                    'Pressure_StdDev': round(stats['Psurf_f_inst_stdDev'], 2),
                    'Specific_Humidity_StdDev': round(stats['Qair_f_inst_stdDev'], 4)
                })
                print(f"Processed: Temp={annual_data[-1]['Temperature_C']}°C, RH={annual_data[-1]['Relative_Humidity']}%")

        except Exception as e:
            print(f"Error processing {year}: {str(e)}")

    return pd.DataFrame(annual_data)

def export_humidity_data():
    """Export both monthly and annual humidity data for Madhya Pradesh"""
    try:
        # Calculate and export monthly data
        print("\nProcessing monthly averages...")
        monthly_df = calculate_monthly_humidity()
        monthly_output = 'mp_monthly_humidity_data.csv'
        monthly_df.to_csv(monthly_output, index=False)
        print(f"Monthly data exported to {monthly_output}")
        print("\nMonthly data sample:")
        print(monthly_df.head())

        # Calculate and export annual data
        print("\nProcessing annual averages...")
        annual_df = calculate_annual_humidity()
        annual_output = 'mp_annual_humidity_data.csv'
        annual_df.to_csv(annual_output, index=False)
        print(f"Annual data exported to {annual_output}")
        print("\nAnnual data:")
        print(annual_df)

        # Generate summary statistics
        print("\nMonthly Summary Statistics:")
        monthly_stats = monthly_df.groupby('Month_Name')['Relative_Humidity'].agg(['mean', 'min', 'max', 'std']).round(2)
        print(monthly_stats)

        print("\nAnnual Summary Statistics:")
        annual_stats = annual_df['Relative_Humidity'].agg(['mean', 'min', 'max', 'std']).round(2)
        print(annual_stats)

    except Exception as e:
        print(f"An error occurred: {str(e)}")
        print("Please check:")
        print("1. Earth Engine authentication is valid")
        print("2. Internet connection is stable")
        print("3. The specified region has available data")

if __name__ == "__main__":
    export_humidity_data()


Processing monthly averages...
Processing monthly data for January 2015...
Number of images: 248
Processed: Temp=16.24°C, RH=51.55%
Processing monthly data for February 2015...
Number of images: 224
Processed: Temp=21.97°C, RH=35.18%
Processing monthly data for March 2015...
Number of images: 248
Processed: Temp=25.2°C, RH=33.94%
Processing monthly data for April 2015...
Number of images: 240
Processed: Temp=31.06°C, RH=24.59%
Processing monthly data for May 2015...
Number of images: 248
Processed: Temp=35.82°C, RH=18.49%
Processing monthly data for June 2015...
Number of images: 240
Processed: Temp=32.77°C, RH=43.2%
Processing monthly data for July 2015...
Number of images: 248
Processed: Temp=28.29°C, RH=68.51%
Processing monthly data for August 2015...
Number of images: 248
Processed: Temp=27.28°C, RH=75.12%
Processing monthly data for September 2015...
Number of images: 240
Processed: Temp=27.58°C, RH=60.53%
Processing monthly data for October 2015...
Number of images: 248
Process

## Export Precipitation

In [ ]:
import ee
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import numpy as np
import calendar

# Initialize Earth Engine
ee.Authenticate()
ee.Initialize(project='ee-geosynta')

def get_mp_boundary():
    """Get Madhya Pradesh's administrative boundary"""
    states = ee.FeatureCollection('FAO/GAUL/2015/level1')
    mp = states.filter(ee.Filter.eq('ADM1_NAME', 'Madhya Pradesh')).geometry()
    return mp

def calculate_monthly_precipitation():
    """Calculate monthly precipitation averages for Madhya Pradesh"""
    chirps = ee.ImageCollection('UCSB-CHG/CHIRPS/PENTAD').select('precipitation')
    roi = get_mp_boundary()

    current_year = datetime.now().year
    current_month = datetime.now().month

    monthly_data = []

    for year in range(2015, current_year + 1):
        for month in range(1, 13):
            if year == current_year and month > current_month:
                continue

            try:
                print(f"Processing monthly data for {calendar.month_name[month]} {year}...")

                monthly_collection = chirps.filter(ee.Filter.calendarRange(year, year, 'year')) \
                                        .filter(ee.Filter.calendarRange(month, month, 'month'))

                num_images = monthly_collection.size().getInfo()
                print(f"Number of images: {num_images}")

                if num_images == 0:
                    print(f"No data available")
                    continue

                # Calculate monthly statistics
                monthly_stats = monthly_collection.reduce(ee.Reducer.mean() \
                    .combine(ee.Reducer.sum(), '', True) \
                    .combine(ee.Reducer.stdDev(), '', True) \
                    .combine(ee.Reducer.minMax(), '', True))

                stats = monthly_stats.reduceRegion(
                    reducer=ee.Reducer.mean(),
                    geometry=roi,
                    scale=5000,
                    maxPixels=1e9
                ).getInfo()

                if stats['precipitation_mean'] is not None:
                    monthly_data.append({
                        'Year': year,
                        'Month': month,
                        'Month_Name': calendar.month_name[month],
                        'Mean_Daily_Precipitation_mm': round(stats['precipitation_mean'], 2),
                        'Total_Monthly_Precipitation_mm': round(stats['precipitation_sum'], 2),
                        'Max_Daily_Precipitation_mm': round(stats['precipitation_max'], 2),
                        'Min_Daily_Precipitation_mm': round(stats['precipitation_min'], 2),
                        'StdDev_Precipitation_mm': round(stats['precipitation_stdDev'], 2)
                    })
                    print(f"Processed: Mean={monthly_data[-1]['Mean_Daily_Precipitation_mm']}mm/day, "
                          f"Total={monthly_data[-1]['Total_Monthly_Precipitation_mm']}mm/month")

            except Exception as e:
                print(f"Error processing {calendar.month_name[month]} {year}: {str(e)}")

    return pd.DataFrame(monthly_data)

def calculate_annual_precipitation():
    """Calculate annual precipitation averages for Madhya Pradesh"""
    chirps = ee.ImageCollection('UCSB-CHG/CHIRPS/PENTAD').select('precipitation')
    roi = get_mp_boundary()

    current_year = datetime.now().year
    annual_data = []

    for year in range(2015, current_year + 1):
        try:
            print(f"Processing annual data for {year}...")

            yearly_collection = chirps.filter(ee.Filter.calendarRange(year, year, 'year'))

            if yearly_collection.size().getInfo() == 0:
                print(f"No data available")
                continue

            # Calculate annual statistics
            yearly_stats = yearly_collection.reduce(ee.Reducer.mean() \
                .combine(ee.Reducer.sum(), '', True) \
                .combine(ee.Reducer.stdDev(), '', True) \
                .combine(ee.Reducer.minMax(), '', True))

            stats = yearly_stats.reduceRegion(
                reducer=ee.Reducer.mean(),
                geometry=roi,
                scale=5000,
                maxPixels=1e9
            ).getInfo()

            if stats['precipitation_mean'] is not None:
                annual_data.append({
                    'Year': year,
                    'Mean_Daily_Precipitation_mm': round(stats['precipitation_mean'], 2),
                    'Total_Annual_Precipitation_mm': round(stats['precipitation_sum'], 2),
                    'Max_Daily_Precipitation_mm': round(stats['precipitation_max'], 2),
                    'Min_Daily_Precipitation_mm': round(stats['precipitation_min'], 2),
                    'StdDev_Precipitation_mm': round(stats['precipitation_stdDev'], 2)
                })
                print(f"Processed: Mean={annual_data[-1]['Mean_Daily_Precipitation_mm']}mm/day, "
                      f"Total={annual_data[-1]['Total_Annual_Precipitation_mm']}mm/year")

        except Exception as e:
            print(f"Error processing {year}: {str(e)}")

    return pd.DataFrame(annual_data)

def analyze_seasonal_patterns(monthly_df):
    """Analyze seasonal patterns in precipitation"""
    # Define seasons (Indian meteorological seasons)
    winter = ['December', 'January', 'February']
    pre_monsoon = ['March', 'April', 'May']
    monsoon = ['June', 'July', 'August', 'September']
    post_monsoon = ['October', 'November']

    monthly_df['Season'] = monthly_df['Month_Name'].apply(lambda x:
        'Winter' if x in winter else
        'Pre-monsoon' if x in pre_monsoon else
        'Monsoon' if x in monsoon else
        'Post-monsoon')

    seasonal_stats = monthly_df.groupby(['Year', 'Season'])['Total_Monthly_Precipitation_mm'].sum().reset_index()
    seasonal_stats = seasonal_stats.rename(columns={'Total_Monthly_Precipitation_mm': 'Total_Seasonal_Precipitation_mm'})

    return seasonal_stats

def export_precipitation_data():
    """Export precipitation data for Madhya Pradesh"""
    try:
        # Calculate and export monthly data
        print("\nProcessing monthly averages...")
        monthly_df = calculate_monthly_precipitation()
        monthly_output = 'mp_monthly_precipitation_data.csv'
        monthly_df.to_csv(monthly_output, index=False)
        print(f"Monthly data exported to {monthly_output}")
        print("\nMonthly data sample:")
        print(monthly_df.head())

        # Calculate and export annual data
        print("\nProcessing annual averages...")
        annual_df = calculate_annual_precipitation()
        annual_output = 'mp_annual_precipitation_data.csv'
        annual_df.to_csv(annual_output, index=False)
        print(f"Annual data exported to {annual_output}")
        print("\nAnnual data:")
        print(annual_df)

        # Calculate and export seasonal patterns
        print("\nAnalyzing seasonal patterns...")
        seasonal_df = analyze_seasonal_patterns(monthly_df)
        seasonal_output = 'mp_seasonal_precipitation_data.csv'
        seasonal_df.to_csv(seasonal_output, index=False)
        print(f"Seasonal data exported to {seasonal_output}")

        # Generate summary statistics
        print("\nMonthly Summary Statistics:")
        monthly_stats = monthly_df.groupby('Month_Name')['Total_Monthly_Precipitation_mm'] \
                                .agg(['mean', 'min', 'max', 'std']).round(2)
        print(monthly_stats)

        print("\nAnnual Summary Statistics:")
        annual_stats = annual_df['Total_Annual_Precipitation_mm'].agg(['mean', 'min', 'max', 'std']).round(2)
        print(annual_stats)

    except Exception as e:
        print(f"An error occurred: {str(e)}")
        print("Please check:")
        print("1. Earth Engine authentication is valid")
        print("2. Internet connection is stable")
        print("3. The specified region has available data")

if __name__ == "__main__":
    export_precipitation_data()


Processing monthly averages...
Processing monthly data for January 2015...
Number of images: 6
Processed: Mean=3.59mm/day, Total=21.52mm/month
Processing monthly data for February 2015...
Number of images: 6
Processed: Mean=1.68mm/day, Total=10.05mm/month
Processing monthly data for March 2015...
Number of images: 6
Processed: Mean=4.53mm/day, Total=27.2mm/month
Processing monthly data for April 2015...
Number of images: 6
Processed: Mean=0.88mm/day, Total=5.25mm/month
Processing monthly data for May 2015...
Number of images: 6
Processed: Mean=0.93mm/day, Total=5.58mm/month
Processing monthly data for June 2015...
Number of images: 6
Processed: Mean=22.57mm/day, Total=135.42mm/month
Processing monthly data for July 2015...
Number of images: 6
Processed: Mean=54.7mm/day, Total=328.17mm/month
Processing monthly data for August 2015...
Number of images: 6
Processed: Mean=46.73mm/day, Total=280.38mm/month
Processing monthly data for September 2015...
Number of images: 6
Processed: Mean=16

## Export Rain Fall

In [ ]:
import ee
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import numpy as np
import calendar

# Initialize Earth Engine
ee.Authenticate()
ee.Initialize(project='ee-geosynta')

def get_mp_boundary():
    """Get Madhya Pradesh's administrative boundary"""
    states = ee.FeatureCollection('FAO/GAUL/2015/level1')
    mp = states.filter(ee.Filter.eq('ADM1_NAME', 'Madhya Pradesh')).geometry()
    return mp

def calculate_monthly_rainfall():
    """Calculate monthly rainfall averages for Madhya Pradesh using GPM V07 data"""
    gpm = ee.ImageCollection('NASA/GPM_L3/IMERG_V07')
    roi = get_mp_boundary()

    current_year = datetime.now().year
    current_month = datetime.now().month

    monthly_data = []

    for year in range(2015, current_year + 1):
        for month in range(1, 13):
            if year == current_year and month > current_month:
                continue

            try:
                print(f"Processing monthly data for {calendar.month_name[month]} {year}...")

                start_date = ee.Date.fromYMD(year, month, 1)
                if month == 12:
                    end_date = ee.Date.fromYMD(year + 1, 1, 1)
                else:
                    end_date = ee.Date.fromYMD(year, month + 1, 1)

                # Get half-hourly precipitation data
                collection = gpm.filter(ee.Filter.date(start_date, end_date)) \
                             .select('precipitation')

                # Get unique dates
                dates = collection.aggregate_array('system:time_start') \
                    .map(lambda t: ee.Date(t).format('YYYY-MM-dd')).distinct()
                dates_list = dates.getInfo()

                daily_totals = []
                rainy_days = 0

                for date in dates_list:
                    # Get all images for this date
                    day_images = collection.filter(
                        ee.Filter.eq('system:time_start', ee.Date.parse('YYYY-MM-dd', date).millis())
                    )

                    # Calculate total precipitation for this day
                    daily_total = day_images.sum().reduceRegion(
                        reducer=ee.Reducer.mean(),
                        geometry=roi,
                        scale=11132,
                        maxPixels=1e9
                    ).get('precipitation').getInfo()

                    # Convert from half-hourly to daily total
                    daily_total = daily_total * 24  # Convert to daily total
                    daily_totals.append(daily_total)

                    # Count as rainy day if daily total > 2.5mm (standard meteorological threshold)
                    if daily_total > 2.5:
                        rainy_days += 1

                if daily_totals:
                    # Calculate monthly statistics
                    monthly_total = sum(daily_totals)
                    daily_mean = monthly_total / len(daily_totals)

                    monthly_data.append({
                        'Year': year,
                        'Month': month,
                        'Month_Name': calendar.month_name[month],
                        'Mean_Daily_Rainfall_mm': round(daily_mean, 2),
                        'Total_Monthly_Rainfall_mm': round(monthly_total, 2),
                        'Max_Daily_Rainfall_mm': round(max(daily_totals), 2),
                        'Min_Daily_Rainfall_mm': round(min(daily_totals), 2),
                        'Rainy_Days': rainy_days,
                        'Days_In_Month': len(daily_totals)
                    })

                    print(f"Processed: Mean={monthly_data[-1]['Mean_Daily_Rainfall_mm']}mm/day, "
                          f"Total={monthly_data[-1]['Total_Monthly_Rainfall_mm']}mm/month, "
                          f"Rainy Days={monthly_data[-1]['Rainy_Days']} out of {monthly_data[-1]['Days_In_Month']} days")

            except Exception as e:
                print(f"Error processing {calendar.month_name[month]} {year}: {str(e)}")

    return pd.DataFrame(monthly_data)

def calculate_annual_rainfall():
    """Calculate annual rainfall averages for Madhya Pradesh"""
    gpm = ee.ImageCollection('NASA/GPM_L3/IMERG_V07')
    roi = get_mp_boundary()

    current_year = datetime.now().year
    annual_data = []

    for year in range(2015, current_year + 1):
        try:
            print(f"Processing annual data for {year}...")

            start_date = ee.Date.fromYMD(year, 1, 1)
            end_date = ee.Date.fromYMD(year + 1, 1, 1)

            # Get precipitation data for the year
            collection = gpm.filter(ee.Filter.date(start_date, end_date)) \
                           .select('precipitation')

            # Get unique dates
            dates = collection.aggregate_array('system:time_start') \
                .map(lambda t: ee.Date(t).format('YYYY-MM-dd')).distinct()
            dates_list = dates.getInfo()

            daily_totals = []
            rainy_days = 0

            for date in dates_list:
                # Get all images for this date
                day_images = collection.filter(
                    ee.Filter.eq('system:time_start', ee.Date.parse('YYYY-MM-dd', date).millis())
                )

                # Calculate total precipitation for this day
                daily_total = day_images.sum().reduceRegion(
                    reducer=ee.Reducer.mean(),
                    geometry=roi,
                    scale=11132,
                    maxPixels=1e9
                ).get('precipitation').getInfo()

                # Convert from half-hourly to daily total
                daily_total = daily_total * 24  # Convert to daily total
                daily_totals.append(daily_total)

                # Count as rainy day if daily total > 2.5mm
                if daily_total > 2.5:
                    rainy_days += 1

            if daily_totals:
                # Calculate annual statistics
                annual_total = sum(daily_totals)
                daily_mean = annual_total / len(daily_totals)

                annual_data.append({
                    'Year': year,
                    'Mean_Daily_Rainfall_mm': round(daily_mean, 2),
                    'Total_Annual_Rainfall_mm': round(annual_total, 2),
                    'Max_Daily_Rainfall_mm': round(max(daily_totals), 2),
                    'Min_Daily_Rainfall_mm': round(min(daily_totals), 2),
                    'Rainy_Days': rainy_days,
                    'Days_In_Year': len(daily_totals),
                    'Monthly_Average_mm': round(annual_total / 12, 2)
                })

                print(f"Processed: Mean={annual_data[-1]['Mean_Daily_Rainfall_mm']}mm/day, "
                      f"Total={annual_data[-1]['Total_Annual_Rainfall_mm']}mm/year, "
                      f"Rainy Days={annual_data[-1]['Rainy_Days']} out of {annual_data[-1]['Days_In_Year']} days")

        except Exception as e:
            print(f"Error processing {year}: {str(e)}")

    return pd.DataFrame(annual_data)

def analyze_monsoon_patterns(monthly_df):
    """Analyze monsoon rainfall patterns"""
    # Define monsoon months (June to September)
    monsoon_months = ['June', 'July', 'August', 'September']

    # Calculate monsoon season totals
    monthly_df['Is_Monsoon'] = monthly_df['Month_Name'].isin(monsoon_months)
    monsoon_stats = monthly_df[monthly_df['Is_Monsoon']].groupby('Year').agg({
        'Total_Monthly_Rainfall_mm': 'sum',
        'Rainy_Days': 'sum',
        'Random_Error_mm': 'mean'
    }).reset_index()

    monsoon_stats = monsoon_stats.rename(columns={
        'Total_Monthly_Rainfall_mm': 'Total_Monsoon_Rainfall_mm',
        'Rainy_Days': 'Monsoon_Rainy_Days',
        'Random_Error_mm': 'Mean_Monsoon_Error_mm'
    })

    return monsoon_stats

def export_rainfall_data():
    """Export rainfall data for Madhya Pradesh"""
    try:
        # Calculate and export monthly data
        print("\nProcessing monthly averages...")
        monthly_df = calculate_monthly_rainfall()
        monthly_output = 'mp_monthly_rainfall_data.csv'
        monthly_df.to_csv(monthly_output, index=False)
        print(f"Monthly data exported to {monthly_output}")
        print("\nMonthly data sample:")
        print(monthly_df.head())

        # Calculate and export annual data
        print("\nProcessing annual averages...")
        annual_df = calculate_annual_rainfall()
        annual_output = 'mp_annual_rainfall_data.csv'
        annual_df.to_csv(annual_output, index=False)
        print(f"Annual data exported to {annual_output}")
        print("\nAnnual data:")
        print(annual_df)

        # Calculate and export monsoon patterns
        print("\nAnalyzing monsoon patterns...")
        monsoon_df = analyze_monsoon_patterns(monthly_df)
        monsoon_output = 'mp_monsoon_rainfall_data.csv'
        monsoon_df.to_csv(monsoon_output, index=False)
        print(f"Monsoon data exported to {monsoon_output}")

        # Generate summary statistics
        print("\nMonthly Summary Statistics:")
        monthly_stats = monthly_df.groupby('Month_Name')['Total_Monthly_Rainfall_mm'] \
                                .agg(['mean', 'min', 'max', 'std']).round(2)
        print(monthly_stats)

        print("\nAnnual Summary Statistics:")
        annual_stats = annual_df['Total_Annual_Rainfall_mm'].agg(['mean', 'min', 'max', 'std']).round(2)
        print(annual_stats)

    except Exception as e:
        print(f"An error occurred: {str(e)}")
        print("Please check:")
        print("1. Earth Engine authentication is valid")
        print("2. Internet connection is stable")
        print("3. The specified region has available data")

if __name__ == "__main__":
    export_rainfall_data()


Processing monthly averages...
Processing monthly data for January 2015...
Processed: Mean=0.85mm/day, Total=26.35mm/month, Rainy Days=4 out of 31 days
Processing monthly data for February 2015...
Processed: Mean=0.29mm/day, Total=8.0mm/month, Rainy Days=0 out of 28 days
Processing monthly data for March 2015...
Processed: Mean=1.99mm/day, Total=61.73mm/month, Rainy Days=6 out of 31 days
Processing monthly data for April 2015...
Processed: Mean=0.26mm/day, Total=7.93mm/month, Rainy Days=0 out of 30 days
Processing monthly data for May 2015...
Processed: Mean=0.35mm/day, Total=10.89mm/month, Rainy Days=1 out of 31 days
Processing monthly data for June 2015...
Processed: Mean=3.38mm/day, Total=101.47mm/month, Rainy Days=9 out of 30 days
Processing monthly data for July 2015...
Processed: Mean=12.82mm/day, Total=397.46mm/month, Rainy Days=19 out of 31 days
Processing monthly data for August 2015...
Processed: Mean=7.39mm/day, Total=229.07mm/month, Rainy Days=16 out of 31 days
Processing 

Processed: Mean=0.09mm/day, Total=2.78mm/month, Rainy Days=0 out of 30 days
Processing monthly data for December 2015...


Processed: Mean=0.01mm/day, Total=0.29mm/month, Rainy Days=0 out of 31 days
Processing monthly data for January 2016...
Processed: Mean=0.55mm/day, Total=16.99mm/month, Rainy Days=2 out of 31 days
Processing monthly data for February 2016...
Processed: Mean=0.04mm/day, Total=1.14mm/month, Rainy Days=0 out of 29 days
Processing monthly data for March 2016...
Processed: Mean=0.28mm/day, Total=8.55mm/month, Rainy Days=1 out of 31 days
Processing monthly data for April 2016...
Processed: Mean=0.03mm/day, Total=0.99mm/month, Rainy Days=0 out of 30 days
Processing monthly data for May 2016...
Processed: Mean=0.17mm/day, Total=5.16mm/month, Rainy Days=0 out of 31 days
Processing monthly data for June 2016...
Processed: Mean=2.45mm/day, Total=73.54mm/month, Rainy Days=6 out of 30 days
Processing monthly data for July 2016...
Processed: Mean=13.66mm/day, Total=423.48mm/month, Rainy Days=24 out of 31 days
Processing monthly data for August 2016...
Processed: Mean=13.55mm/day, Total=420.08mm/mont

## Export Wind Speed

In [ ]:
import ee
import pandas as pd
from datetime import datetime
import calendar
import numpy as np

# Initialize Earth Engine
ee.Authenticate()
ee.Initialize(project='ee-geosynta')

def get_mp_boundary():
    """Get Madhya Pradesh's administrative boundary"""
    states = ee.FeatureCollection('FAO/GAUL/2015/level1')
    mp = states.filter(ee.Filter.eq('ADM1_NAME', 'Madhya Pradesh')).geometry()
    return mp

def calculate_monthly_wind():
    """Calculate monthly wind statistics for Madhya Pradesh using ERA5 data"""
    # ERA5 provides u and v components of wind at 10m height
    era5 = ee.ImageCollection('ECMWF/ERA5_LAND/HOURLY')
    roi = get_mp_boundary()

    current_year = datetime.now().year
    current_month = datetime.now().month

    monthly_data = []

    for year in range(2015, current_year + 1):
        for month in range(1, 13):
            if year == current_year and month > current_month:
                continue

            try:
                print(f"Processing monthly wind data for {calendar.month_name[month]} {year}...")

                start_date = ee.Date.fromYMD(year, month, 1)
                if month == 12:
                    end_date = ee.Date.fromYMD(year + 1, 1, 1)
                else:
                    end_date = ee.Date.fromYMD(year, month + 1, 1)

                # Get wind components
                collection = era5.filter(ee.Filter.date(start_date, end_date)) \
                               .select(['u_component_of_wind_10m', 'v_component_of_wind_10m'])

                # Get unique dates
                dates = collection.aggregate_array('system:time_start') \
                    .map(lambda t: ee.Date(t).format('YYYY-MM-dd')).distinct()
                dates_list = dates.getInfo()

                daily_speeds = []
                daily_directions = []

                for date in dates_list:
                    # Get all images for this date
                    day_images = collection.filter(
                        ee.Filter.eq('system:time_start', ee.Date.parse('YYYY-MM-dd', date).millis())
                    )

                    # Calculate daily mean wind components
                    daily_means = day_images.mean().reduceRegion(
                        reducer=ee.Reducer.mean(),
                        geometry=roi,
                        scale=11132,
                        maxPixels=1e9
                    ).getInfo()

                    # Extract u and v components
                    u = daily_means['u_component_of_wind_10m']
                    v = daily_means['v_component_of_wind_10m']

                    # Calculate wind speed and direction
                    speed = np.sqrt(u**2 + v**2)
                    direction = (270 - np.degrees(np.arctan2(v, u))) % 360  # Convert to meteorological convention

                    daily_speeds.append(speed)
                    daily_directions.append(direction)

                if daily_speeds:
                    # Calculate monthly statistics
                    mean_speed = np.mean(daily_speeds)
                    max_speed = np.max(daily_speeds)
                    min_speed = np.min(daily_speeds)

                    # Calculate prevailing wind direction (most common direction sector)
                    sectors = np.array([0, 45, 90, 135, 180, 225, 270, 315, 360])
                    digitized = np.digitize(daily_directions, sectors)
                    direction_counts = np.bincount(digitized)
                    prevailing_sector = sectors[np.argmax(direction_counts[1:])]  # Skip first bin

                    # Count high wind days (speed > 5 m/s)
                    high_wind_days = sum(1 for speed in daily_speeds if speed > 5)

                    monthly_data.append({
                        'Year': year,
                        'Month': month,
                        'Month_Name': calendar.month_name[month],
                        'Mean_Wind_Speed_ms': round(mean_speed, 2),
                        'Max_Wind_Speed_ms': round(max_speed, 2),
                        'Min_Wind_Speed_ms': round(min_speed, 2),
                        'Prevailing_Wind_Direction_deg': round(prevailing_sector),
                        'High_Wind_Days': high_wind_days,
                        'Days_In_Month': len(daily_speeds)
                    })

                    print(f"Processed: Mean Speed={monthly_data[-1]['Mean_Wind_Speed_ms']}m/s, "
                          f"Prevailing Direction={monthly_data[-1]['Prevailing_Wind_Direction_deg']}°, "
                          f"High Wind Days={monthly_data[-1]['High_Wind_Days']}")

            except Exception as e:
                print(f"Error processing {calendar.month_name[month]} {year}: {str(e)}")

    return pd.DataFrame(monthly_data)

def calculate_annual_wind():
    """Calculate annual wind statistics for Madhya Pradesh"""
    era5 = ee.ImageCollection('ECMWF/ERA5_LAND/HOURLY')
    roi = get_mp_boundary()

    current_year = datetime.now().year
    annual_data = []

    for year in range(2015, current_year + 1):
        try:
            print(f"Processing annual wind data for {year}...")

            start_date = ee.Date.fromYMD(year, 1, 1)
            end_date = ee.Date.fromYMD(year + 1, 1, 1)

            # Get wind components
            collection = era5.filter(ee.Filter.date(start_date, end_date)) \
                           .select(['u_component_of_wind_10m', 'v_component_of_wind_10m'])

            # Get unique dates
            dates = collection.aggregate_array('system:time_start') \
                .map(lambda t: ee.Date(t).format('YYYY-MM-dd')).distinct()
            dates_list = dates.getInfo()

            daily_speeds = []
            daily_directions = []
            monthly_prevailing_directions = []

            for date in dates_list:
                # Get all images for this date
                day_images = collection.filter(
                    ee.Filter.eq('system:time_start', ee.Date.parse('YYYY-MM-dd', date).millis())
                )

                # Calculate daily mean wind components
                daily_means = day_images.mean().reduceRegion(
                    reducer=ee.Reducer.mean(),
                    geometry=roi,
                    scale=11132,
                    maxPixels=1e9
                ).getInfo()

                # Extract u and v components
                u = daily_means['u_component_of_wind_10m']
                v = daily_means['v_component_of_wind_10m']

                # Calculate wind speed and direction
                speed = np.sqrt(u**2 + v**2)
                direction = (270 - np.degrees(np.arctan2(v, u))) % 360

                daily_speeds.append(speed)
                daily_directions.append(direction)

            if daily_speeds:
                # Calculate annual statistics
                mean_speed = np.mean(daily_speeds)
                max_speed = np.max(daily_speeds)
                min_speed = np.min(daily_speeds)

                # Calculate annual prevailing wind direction
                sectors = np.array([0, 45, 90, 135, 180, 225, 270, 315, 360])
                digitized = np.digitize(daily_directions, sectors)
                direction_counts = np.bincount(digitized)
                prevailing_sector = sectors[np.argmax(direction_counts[1:])]

                # Count high wind days (speed > 5 m/s)
                high_wind_days = sum(1 for speed in daily_speeds if speed > 5)

                annual_data.append({
                    'Year': year,
                    'Mean_Wind_Speed_ms': round(mean_speed, 2),
                    'Max_Wind_Speed_ms': round(max_speed, 2),
                    'Min_Wind_Speed_ms': round(min_speed, 2),
                    'Prevailing_Wind_Direction_deg': round(prevailing_sector),
                    'High_Wind_Days': high_wind_days,
                    'Days_In_Year': len(daily_speeds),
                    'Monthly_Average_Speed_ms': round(mean_speed, 2)
                })

                print(f"Processed: Mean Speed={annual_data[-1]['Mean_Wind_Speed_ms']}m/s, "
                      f"Prevailing Direction={annual_data[-1]['Prevailing_Wind_Direction_deg']}°, "
                      f"High Wind Days={annual_data[-1]['High_Wind_Days']} out of {annual_data[-1]['Days_In_Year']} days")

        except Exception as e:
            print(f"Error processing {year}: {str(e)}")

    return pd.DataFrame(annual_data)

def export_wind_data():
    """Export wind data analysis"""
    try:
        # Calculate and export monthly data
        print("\nProcessing monthly wind averages...")
        monthly_df = calculate_monthly_wind()
        monthly_output = 'mp_monthly_wind_data.csv'
        monthly_df.to_csv(monthly_output, index=False)
        print(f"Monthly wind data exported to {monthly_output}")

        # Calculate and export annual data
        print("\nProcessing annual wind averages...")
        annual_df = calculate_annual_wind()
        annual_output = 'mp_annual_wind_data.csv'
        annual_df.to_csv(annual_output, index=False)
        print(f"Annual wind data exported to {annual_output}")

        # Generate summary statistics
        print("\nMonthly Wind Summary Statistics:")
        monthly_stats = monthly_df.groupby('Month_Name')['Mean_Wind_Speed_ms'].agg(['mean', 'min', 'max']).round(2)
        print(monthly_stats)

        print("\nAnnual Wind Summary Statistics:")
        annual_stats = annual_df['Mean_Wind_Speed_ms'].agg(['mean', 'min', 'max']).round(2)
        print(annual_stats)

    except Exception as e:
        print(f"An error occurred: {str(e)}")

if __name__ == "__main__":
    export_wind_data()


Processing monthly wind averages...
Processing monthly wind data for January 2015...
Processed: Mean Speed=1.2m/s, Prevailing Direction=45°, High Wind Days=0
Processing monthly wind data for February 2015...
Processed: Mean Speed=1.21m/s, Prevailing Direction=45°, High Wind Days=0
Processing monthly wind data for March 2015...
Processed: Mean Speed=1.01m/s, Prevailing Direction=90°, High Wind Days=0
Processing monthly wind data for April 2015...
Processed: Mean Speed=1.26m/s, Prevailing Direction=225°, High Wind Days=0
Processing monthly wind data for May 2015...
Processed: Mean Speed=1.86m/s, Prevailing Direction=225°, High Wind Days=0
Processing monthly wind data for June 2015...
Processed: Mean Speed=2.03m/s, Prevailing Direction=225°, High Wind Days=0
Processing monthly wind data for July 2015...
Processed: Mean Speed=2.87m/s, Prevailing Direction=225°, High Wind Days=0
Processing monthly wind data for August 2015...
Processed: Mean Speed=2.06m/s, Prevailing Direction=225°, High W

## Export Solar Radiation

In [ ]:
import ee
import pandas as pd
from datetime import datetime
import calendar
import numpy as np

# Initialize Earth Engine
ee.Authenticate()
ee.Initialize(project='ee-geosynta')

def get_mp_boundary():
    """Get Madhya Pradesh's administrative boundary"""
    states = ee.FeatureCollection('FAO/GAUL/2015/level1')
    mp = states.filter(ee.Filter.eq('ADM1_NAME', 'Madhya Pradesh')).geometry()
    return mp

def calculate_monthly_solar():
    """Calculate monthly solar radiation statistics for Madhya Pradesh using ERA5 data"""
    era5 = ee.ImageCollection('ECMWF/ERA5_LAND/HOURLY')
    roi = get_mp_boundary()

    current_year = datetime.now().year
    current_month = datetime.now().month

    monthly_data = []

    # Solar radiation variables from ERA5-Land
    solar_vars = [
        'surface_solar_radiation_downwards',  # Total solar radiation
        'surface_direct_solar_radiation',     # Direct solar radiation
        'surface_net_solar_radiation'         # Net solar radiation
    ]

    for year in range(2015, current_year + 1):
        for month in range(1, 13):
            if year == current_year and month > current_month:
                continue

            try:
                print(f"Processing monthly solar data for {calendar.month_name[month]} {year}...")

                start_date = ee.Date.fromYMD(year, month, 1)
                if month == 12:
                    end_date = ee.Date.fromYMD(year + 1, 1, 1)
                else:
                    end_date = ee.Date.fromYMD(year, month + 1, 1)

                # Get solar radiation data
                collection = era5.filter(ee.Filter.date(start_date, end_date)) \
                               .select(solar_vars)

                # Get unique dates
                dates = collection.aggregate_array('system:time_start') \
                    .map(lambda t: ee.Date(t).format('YYYY-MM-dd')).distinct()
                dates_list = dates.getInfo()

                daily_total = []
                daily_direct = []
                daily_net = []
                sunny_days = 0  # Days with high solar radiation

                for date in dates_list:
                    # Get all images for this date
                    day_images = collection.filter(
                        ee.Filter.eq('system:time_start', ee.Date.parse('YYYY-MM-dd', date).millis())
                    )

                    # Calculate daily accumulated radiation
                    daily_sums = day_images.sum().reduceRegion(
                        reducer=ee.Reducer.mean(),
                        geometry=roi,
                        scale=11132,
                        maxPixels=1e9
                    ).getInfo()

                    # Convert from J/m² to kWh/m²/day (divide by 3.6e6)
                    total_rad = daily_sums['surface_solar_radiation_downwards'] / 3.6e6
                    direct_rad = daily_sums['surface_direct_solar_radiation'] / 3.6e6
                    net_rad = daily_sums['surface_net_solar_radiation'] / 3.6e6

                    daily_total.append(total_rad)
                    daily_direct.append(direct_rad)
                    daily_net.append(net_rad)

                    # Count sunny days (total radiation > 5 kWh/m²/day)
                    if total_rad > 5:
                        sunny_days += 1

                if daily_total:
                    # Calculate monthly statistics
                    monthly_data.append({
                        'Year': year,
                        'Month': month,
                        'Month_Name': calendar.month_name[month],
                        'Mean_Daily_Total_Radiation_kWh_m2': round(np.mean(daily_total), 2),
                        'Max_Daily_Total_Radiation_kWh_m2': round(np.max(daily_total), 2),
                        'Min_Daily_Total_Radiation_kWh_m2': round(np.min(daily_total), 2),
                        'Mean_Daily_Direct_Radiation_kWh_m2': round(np.mean(daily_direct), 2),
                        'Mean_Daily_Net_Radiation_kWh_m2': round(np.mean(daily_net), 2),
                        'Monthly_Total_Radiation_kWh_m2': round(sum(daily_total), 2),
                        'Sunny_Days': sunny_days,
                        'Days_In_Month': len(daily_total)
                    })

                    print(f"Processed: Mean Total={monthly_data[-1]['Mean_Daily_Total_Radiation_kWh_m2']}kWh/m²/day, "
                          f"Monthly Total={monthly_data[-1]['Monthly_Total_Radiation_kWh_m2']}kWh/m², "
                          f"Sunny Days={monthly_data[-1]['Sunny_Days']}")

            except Exception as e:
                print(f"Error processing {calendar.month_name[month]} {year}: {str(e)}")

    return pd.DataFrame(monthly_data)

def calculate_annual_solar():
    """Calculate annual solar radiation statistics for Madhya Pradesh"""
    era5 = ee.ImageCollection('ECMWF/ERA5_LAND/HOURLY')
    roi = get_mp_boundary()

    current_year = datetime.now().year
    annual_data = []

    solar_vars = [
        'surface_solar_radiation_downwards',
        'surface_direct_solar_radiation',
        'surface_net_solar_radiation'
    ]

    for year in range(2015, current_year + 1):
        try:
            print(f"Processing annual solar data for {year}...")

            start_date = ee.Date.fromYMD(year, 1, 1)
            end_date = ee.Date.fromYMD(year + 1, 1, 1)

            # Get solar radiation data
            collection = era5.filter(ee.Filter.date(start_date, end_date)) \
                           .select(solar_vars)

            # Get unique dates
            dates = collection.aggregate_array('system:time_start') \
                .map(lambda t: ee.Date(t).format('YYYY-MM-dd')).distinct()
            dates_list = dates.getInfo()

            daily_total = []
            daily_direct = []
            daily_net = []
            sunny_days = 0

            for date in dates_list:
                # Get all images for this date
                day_images = collection.filter(
                    ee.Filter.eq('system:time_start', ee.Date.parse('YYYY-MM-dd', date).millis())
                )

                # Calculate daily accumulated radiation
                daily_sums = day_images.sum().reduceRegion(
                    reducer=ee.Reducer.mean(),
                    geometry=roi,
                    scale=11132,
                    maxPixels=1e9
                ).getInfo()

                # Convert from J/m² to kWh/m²/day
                total_rad = daily_sums['surface_solar_radiation_downwards'] / 3.6e6
                direct_rad = daily_sums['surface_direct_solar_radiation'] / 3.6e6
                net_rad = daily_sums['surface_net_solar_radiation'] / 3.6e6

                daily_total.append(total_rad)
                daily_direct.append(direct_rad)
                daily_net.append(net_rad)

                # Count sunny days
                if total_rad > 5:
                    sunny_days += 1

            if daily_total:
                # Calculate annual statistics
                annual_data.append({
                    'Year': year,
                    'Mean_Daily_Total_Radiation_kWh_m2': round(np.mean(daily_total), 2),
                    'Max_Daily_Total_Radiation_kWh_m2': round(np.max(daily_total), 2),
                    'Min_Daily_Total_Radiation_kWh_m2': round(np.min(daily_total), 2),
                    'Mean_Daily_Direct_Radiation_kWh_m2': round(np.mean(daily_direct), 2),
                    'Mean_Daily_Net_Radiation_kWh_m2': round(np.mean(daily_net), 2),
                    'Annual_Total_Radiation_kWh_m2': round(sum(daily_total), 2),
                    'Sunny_Days': sunny_days,
                    'Days_In_Year': len(daily_total),
                    'Monthly_Average_Radiation_kWh_m2': round(sum(daily_total)/12, 2)
                })

                print(f"Processed: Mean Total={annual_data[-1]['Mean_Daily_Total_Radiation_kWh_m2']}kWh/m²/day, "
                      f"Annual Total={annual_data[-1]['Annual_Total_Radiation_kWh_m2']}kWh/m², "
                      f"Sunny Days={annual_data[-1]['Sunny_Days']}")

        except Exception as e:
            print(f"Error processing {year}: {str(e)}")

    return pd.DataFrame(annual_data)

def export_solar_data():
    """Export solar radiation analysis data"""
    try:
        # Calculate and export monthly data
        print("\nProcessing monthly solar averages...")
        monthly_df = calculate_monthly_solar()
        monthly_output = 'mp_monthly_solar_data.csv'
        monthly_df.to_csv(monthly_output, index=False)
        print(f"Monthly solar data exported to {monthly_output}")

        # Calculate and export annual data
        print("\nProcessing annual solar averages...")
        annual_df = calculate_annual_solar()
        annual_output = 'mp_annual_solar_data.csv'
        annual_df.to_csv(annual_output, index=False)
        print(f"Annual solar data exported to {annual_output}")

        # Generate summary statistics
        print("\nMonthly Solar Radiation Summary Statistics:")
        monthly_stats = monthly_df.groupby('Month_Name')['Mean_Daily_Total_Radiation_kWh_m2'] \
                                .agg(['mean', 'min', 'max']).round(2)
        print(monthly_stats)

        print("\nAnnual Solar Radiation Summary Statistics:")
        annual_stats = annual_df['Mean_Daily_Total_Radiation_kWh_m2'].agg(['mean', 'min', 'max']).round(2)
        print(annual_stats)

    except Exception as e:
        print(f"An error occurred: {str(e)}")

if __name__ == "__main__":
    export_solar_data()